In [6]:
# # Gets the rows of select line feature
# def selected_features(layer):
#     # Check if there is a selection
#     if arcpy.Describe(layer).FIDSet:
#         # Count the selected features
#         result = arcpy.GetCount_management(layer)   
#         count = int(result.getOutput(0))
#         fids = arcpy.Describe(layer).FIDSet.split('; ')
#     else:
#         # No selection, return 0
#         count = 0
#     return fids

# # Usage

# #1. Define input layer
# layer = "CCCLClipProj"
# print(selected_features(layer))
# row_list = selected_features(layer)

['0']


In [3]:
# #generate a query from the selected features
# queryString = ''
# x = 0
# for i in row_list:
#     queryString += "OBJECTID = " + i
#     x = x + 1
#     try:
#         if (row_list[x]):
#             queryString += " Or "
#     except IndexError:
#         print("")

# print(queryString)

﻿
OBJECTID = 0


In [4]:
# jdm: Can we clarify what this step does?
# outFeature = r"G:\My Drive\Projects\FlSeaGrantHurricaneViewsheds\Data\HurricaneIvan"
# arcpy.analysis.Select(
#     in_features= layer,
#     out_feature_class= outFeature,
#     where_clause= queryString
# )

<Result 'G:\\My Drive\\Projects\\FlSeaGrantHurricaneViewsheds\\Data\\HurricaneIvan.shp'>

In [20]:
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/get-raster-properties.htm
rastTop = arcpy.GetRasterProperties_management("121.img", "TOP")
rastLeft = arcpy.GetRasterProperties_management("121.img", "LEFT")
rastBottom = arcpy.GetRasterProperties_management("121.img", "BOTTOM")
rastRight = arcpy.GetRasterProperties_management("121.img", "RIGHT")
rastTop, rastLeft, rastBottom, rastRight

(<Result '500312.12'>, <Result '1140455.35'>, <Result '494602.12'>, <Result '1160295.35'>)

In [23]:
# https://pro.arcgis.com/en/pro-app/latest/arcpy/get-started/writing-geometries.htm
arcpy.env.overwriteOutput = True
coordinates = [
    (500312.12, 1140455.35),
    (494602.12, 1160295.35),
]

# Create a feature class with a spatial reference of GCS WGS 1984
result = arcpy.management.CreateFeatureclass(
    arcpy.env.scratchGDB, "esri_poly", "POLYGON", spatial_reference=6441
)
feature_class = result[0]

# Write feature to new feature class
with arcpy.da.InsertCursor(feature_class, ["SHAPE@"]) as cursor:
    cursor.insertRow([coordinates])

In [10]:
#parameters to modify:
# lineInput = "line_Select" jdm: I believe this can just be the selected line
# in our case the CCCL line. Although if we have an area with multiple lines we can modify 
# script do the detect selection fucntionality
lineInput = "CCCLClipProj"
pointOutput = r"G:\My Drive\Projects\FlSeaGrantHurricaneViewsheds\Data\HurricaneIvan\viewshed.gdb\ObserverPoints"
pointSpacing = 10
pointInput = "GeneratePoints"
rasterInput =  "121.img"
observerElevation = "6 Meters"
viewDistance = "1 Miles"
outRaster = r"G:\My Drive\Projects\FlSeaGrantHurricaneViewsheds\Data\HurricaneIvan\viewshed.gdb\Viewshed_121" #FL_G3??

#Only the part of the CCCL line to that is accounted for w/in the extent of the input raster


# generate points along line
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/generate-points-along-lines.htm
arcpy.management.GeneratePointsAlongLines(
    Input_Features= lineInput,
    Output_Feature_Class= pointOutput,
    Point_Placement="PERCENTAGE",
    Percentage=pointSpacing,
    Include_End_Points="NO_END_POINTS",
    Add_Chainage_Fields="NO_CHAINAGE"
)

<Result 'G:\\My Drive\\Projects\\FlSeaGrantHurricaneViewsheds\\Data\\HurricaneIvan\\viewshed.gdb\\ObserverPoints'>

In [75]:
#geodesic viewshed
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["NAD_1983_UTM_Zone_16N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-87.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]],VERTCS["NAD_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PARAMETER["Vertical_Shift",0.0],PARAMETER["Direction",1.0],UNIT["Meter",1.0]]', parallelProcessingFactor="100%", scratchWorkspace=r"E:\Projects\viewshed.gdb"):
    out_raster = arcpy.sa.Viewshed2(
        in_raster=rasterInput,
        in_observer_features=pointInput,
        out_agl_raster=None,
        analysis_type="FREQUENCY",
        vertical_error="0 Meters",
        out_observer_region_relationship_table=None,
        refractivity_coefficient=0.13,
        surface_offset="0 Meters",
        observer_elevation=None,
        observer_offset=observerElevation,
        inner_radius=None,
        inner_radius_is_3d="GROUND",
        outer_radius=viewDistance,
        outer_radius_is_3d="GROUND",
        horizontal_start_angle=0,
        horizontal_end_angle=360,
        vertical_upper_angle=180,
        vertical_lower_angle=-180,
        analysis_method="ALL_SIGHTLINES",
        analysis_target_device="CPU_ONLY"
    )
    out_raster.save(outRaster)